***
***
# __Update 3: CSC 4220__
### *Adam Becklehimer & David Wallace*
***
***

In [2]:
from __future__ import division
import concurrent.futures
import pandas as pd
import numpy as np
import re
from scipy import stats



from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_fscore_support
from joblib import Parallel, delayed
import multiprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D

Using TensorFlow backend.


In [4]:
from collections import Counter
from imblearn.over_sampling import SMOTE

***
***
## Helper Scripts
***
***


In [5]:
def runs(lst):
	#"Iterator, chunks repeated values"
    for j,two in enumerate(lst):
        if j == 0:
            one,i = two,0
        if one!=two:
            yield j - i,one
            i = j
        one=two
    yield j - i + 1,two


def cliffsDelta(lst1,lst2,dull = [0.147, 0.33, 0.474, ][0] ):
    #"Returns true if there are more than 'dull' differences"
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats,x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] <  x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
        d= (more - less) / (m*n)
	# return abs(d)  > dull
    return round(abs(d), 3) ### return the clacluated cliffs delat
	

def _cliffsDelta():
	#"demo function"
    lst1=[1,2,3,4,5,6,7]
    for r in [1.01,1.1,1.21, 1.5, 2]:
        lst2=map(lambda x: x*r,lst1)
        print (r,cliffsDelta(lst1,lst2)) # should return False

def compareLists(defective_vals_for_feature, non_defective_vals_for_feature):

        print ('='*25)
        print ("Defective values [MEDIAN]:{}, [MEAN]:{}, [COUNT]:{}".format(np.median(list(defective_vals_for_feature)), np.mean(list(defective_vals_for_feature)), len(defective_vals_for_feature)))
        print ("Non Defective values [MEDIAN]:{}, [MEAN]:{}, [COUNT]:{}".format(np.median(list(non_defective_vals_for_feature)), np.mean(list(non_defective_vals_for_feature)), len(non_defective_vals_for_feature)))
        try:
            TS, p = stats.mannwhitneyu(list(defective_vals_for_feature), list(non_defective_vals_for_feature), alternative='greater')
        except ValueError:
            TS, p = 0.0, 1.0
        cliffs_delta = cliffsDelta(list(defective_vals_for_feature), list(non_defective_vals_for_feature))
        print ('*'*25)
        #print ('Feature:{}, pee value:{}, cliffs:{}'.format(feature_, p, cliffs_delta))
        print ('pee value:{}, cliffs:{}'.format(p, cliffs_delta))
        print ('*'*25)
        print ('='*50)

In [8]:
executor = concurrent.futures.ThreadPoolExecutor(max_workers=28)

def read():
    df_ = pd.read_csv('../../Update2/update2/update2.csv')
    #df_ = pd.read_csv('~/CSC4220/update2/update2.csv', engine = 'python')
    return df_

a = executor.submit(read)

print(a)

<Future at 0x1a412aae80 state=running>


In [10]:
print(a)

<Future at 0x1a412aae80 state=finished returned DataFrame>


***
***
# !!! Wait Here Until DataFrame returned !!!
***
***

In [11]:
df_ = a.result()
#print(df_)

df_.REPO_TYPE.unique()

array(['ComputationalChemistry', 'Astronomy', 'ComputationalBiology',
       'ComputationalScience', 'ComputationalIntelligence'], dtype=object)

In [12]:
df_comp_bio = (df_[df_['REPO_TYPE']== 'ComputationalBiology'])
df_astronomy = (df_[df_['REPO_TYPE'] == 'Astronomy'])
df_comp_chem = (df_[df_['REPO_TYPE'] == 'ComputationalChemistry'])
df_comp_sci = (df_[df_['REPO_TYPE'] == 'ComputationalScience'])
df_comp_intel = (df_[df_['REPO_TYPE'] == 'ComputationalIntelligence'])
df_others = pd.concat([df_comp_sci, df_comp_intel], axis=0)


df_comp_bio_full = df_comp_bio[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT" , "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_astro_full = df_astronomy[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT",  "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_comp_chem_full = df_comp_chem[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT" , "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_others_full = df_others[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT",  "CHANGE_FILE_CNT", "SECU_FLAG"]]

***
***
## Mann - Whitney U Test and Effect Size
***
***

In [13]:
def Mann (model,feature):
    valid_features =[]
    invalid_features =[]
    for index, row in model.iterrows():
        if(row['SECU_FLAG'] == "NEUTRAL"):
            valid_features.append(row[feature])
        else:
            invalid_features.append(row[feature])
   
    #print(str(model))
    print("  ")
    print( feature)
    print("\n")
    compareLists(invalid_features,valid_features)
    return 0

In [14]:
print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"ADD_LOC")

print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"PRIOR_AGE")


print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEL_LOC")



print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"TOT_LOC")


print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEV_EXP")


print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEV_RECENT")




print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"CHANGE_FILE_CNT")


MAN WHITNEY U TEST FOR df_others
  
ADD_LOC


Defective values [MEDIAN]:49.0, [MEAN]:360.4206008583691, [COUNT]:233
Non Defective values [MEDIAN]:23.0, [MEAN]:593.3756552125801, [COUNT]:5151
*************************
pee value:5.4656250954923225e-06, cliffs:0.17
*************************
MAN WHITNEY U TEST FOR df_comp_bio_full
  
ADD_LOC


Defective values [MEDIAN]:16.0, [MEAN]:524.2652541833684, [COUNT]:15717
Non Defective values [MEDIAN]:11.0, [MEAN]:663.8411968646367, [COUNT]:159216
*************************
pee value:7.19411106581891e-82, cliffs:0.092
*************************
MAN WHITNEY U TEST FOR df_astro_full
  
ADD_LOC


Defective values [MEDIAN]:14.0, [MEAN]:146.26299504950495, [COUNT]:4848
Non Defective values [MEDIAN]:11.0, [MEAN]:307.9216749871992, [COUNT]:62496
*************************
pee value:2.4756092002005906e-16, cliffs:0.07
*************************
MAN WHITNEY U TEST FOR df_comp_chem_full
  
ADD_LOC


Defective values [MEDIAN]:28.0, [MEAN]:1583.1567711807872, [CO

0

***
***
# General Transfer Learning
### Just Setup Stuff Here
***
***

In [12]:
def run_transfer_model(model, train_X, train_y, target_X, target_y):
    
    train_y = np.ravel(train_y)
    target_y = np.ravel(target_y)
    
    model.fit(train_X, train_y)
    y_predi = model.predict(target_X)
    print(precision_recall_fscore_support(target_y, y_predi,average = None))
    print(confusion_matrix(target_y, y_predi))

In [13]:
Bio_Xy = df_comp_bio_full.to_numpy()
Bio_X = Bio_Xy[:,0:7]
Bio_y = Bio_Xy[:,7]

Astro_Xy = df_astro_full.to_numpy()
Astro_X = Astro_Xy[:,0:7]
Astro_y = Astro_Xy[:,7]

Chem_Xy = df_comp_chem_full.to_numpy()
Chem_X = Chem_Xy[:,0:7]
Chem_y = Chem_Xy[:,7]

Others_Xy = df_others_full.to_numpy()
Others_X = Others_Xy[:,0:7]
Others_y = Others_Xy[:,7]


#le = preprocessing.LabelEncoder()
#
#Bio_y = le.fit_transform(Bio_y)
#Astro_y = le.fit_transform(Astro_y)

Bio_y = label_binarize(np.ravel(Bio_y), classes=['NEUTRAL', 'INSECURE'])
Astro_y = label_binarize(np.ravel(Astro_y), classes=['NEUTRAL', 'INSECURE'])
Chem_y = label_binarize(np.ravel(Chem_y), classes=['NEUTRAL', 'INSECURE'])
Others_y = label_binarize(np.ravel(Others_y), classes=['NEUTRAL', 'INSECURE'])

print(Bio_y.shape, Astro_y.shape, Chem_y.shape, Others_y.shape)
print(Others_y)

(174933, 1) (67344, 1) (52395, 1) (5384, 1)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


***
***
# SMOTE OPs
***
***

In [14]:
sm = SMOTE(random_state=42)
Bio_X_res, Bio_y_res = sm.fit_resample(Bio_X, np.ravel(Bio_y))
Astro_X_res, Astro_y_res = sm.fit_resample(Astro_X, np.ravel(Astro_y))
Chem_X_res, Chem_y_res = sm.fit_resample(Chem_X, np.ravel(Chem_y))
Others_X_res, Others_y_res = sm.fit_resample(Others_X, np.ravel(Others_y))

***
***
# Deep Learning With Keras
***
***

In [62]:
model = Sequential()
model.add(Dense(15, input_dim=7, activation='relu'))
#model.add(Conv1D(4, kernel_size=2, activation='relu'))
#model.add(Dropout(0.50))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(75, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(15, activation='relu'))
#model.add(Dropout(0.125))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
sc = StandardScaler()
X = sc.fit_transform(Bio_X_res)

model.fit(X, Bio_y_res, epochs=1, batch_size=25)

predictions = model.predict_classes(Astro_X)

print(precision_recall_fscore_support(Astro_y, predictions,average = None))
print(confusion_matrix(Astro_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Chem_X)

print(precision_recall_fscore_support(Chem_y, predictions,average = None))
print(confusion_matrix(Chem_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Others_X)

print(precision_recall_fscore_support(Others_y, predictions,average = None))
print(confusion_matrix(Others_y, predictions))

Epoch 1/1
318432/318432 [==============================] - 16s 51us/step - loss: 0.6212 - accuracy: 0.6374
(array([0.92382768, 0.06592207]), array([0.58917371, 0.37376238]), array([0.71949039, 0.1120767 ]), array([62496,  4848]))
[[36821 25675]
 [ 3036  1812]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.94293203, 0.05773092]), array([0.77201045, 0.23015344]), array([0.8489537 , 0.09230769]), array([49397,  2998]))
[[38135 11262]
 [ 2308   690]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.95735805, 0.04583723]), array([0.8019802 , 0.21030043]), array([0.87280794, 0.07526882]), array([5151,  233]))
[[4131 1020]
 [ 184   49]]


In [64]:
sc = StandardScaler()
X = sc.fit_transform(Astro_X_res)

model.fit(X, Astro_y_res, epochs=1, batch_size=25)

predictions = model.predict_classes(Bio_X)

print(precision_recall_fscore_support(Bio_y, predictions,average = None))
print(confusion_matrix(Bio_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Chem_X)

print(precision_recall_fscore_support(Chem_y, predictions,average = None))
print(confusion_matrix(Chem_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Others_X)

print(precision_recall_fscore_support(Others_y, predictions,average = None))
print(confusion_matrix(Others_y, predictions))

Epoch 1/1
124992/124992 [==============================] - 6s 50us/step - loss: 0.6848 - accuracy: 0.5427
(array([0.91013896, 0.08941453]), array([0.96590795, 0.03391232]), array([0.93719453, 0.04917428]), array([159216,  15717]))
[[153788   5428]
 [ 15184    533]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.94281498, 0.05768164]), array([0.9312104 , 0.06937959]), array([0.93697676, 0.06299213]), array([49397,  2998]))
[[45999  3398]
 [ 2790   208]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.95675355, 0.04375   ]), array([0.94059406, 0.06008584]), array([0.94860499, 0.05063291]), array([5151,  233]))
[[4845  306]
 [ 219   14]]


In [65]:
sc = StandardScaler()
X = sc.fit_transform(Chem_X_res)

model.fit(X, Chem_y_res, epochs=1, batch_size=10)

predictions = model.predict_classes(Bio_X)

print(precision_recall_fscore_support(Bio_y, predictions,average = None))
print(confusion_matrix(Bio_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Astro_X)

print(precision_recall_fscore_support(Astro_y, predictions,average = None))
print(confusion_matrix(Astro_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
predictions = model.predict_classes(Others_X)

print(precision_recall_fscore_support(Others_y, predictions,average = None))
print(confusion_matrix(Others_y, predictions))

Epoch 1/1
98794/98794 [==============================] - 12s 122us/step - loss: 0.6696 - accuracy: 0.5886
(array([0.91010084, 0.05921053]), array([0.9982037 , 0.00114526]), array([0.9521185 , 0.00224705]), array([159216,  15717]))
[[158930    286]
 [ 15699     18]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.92794935, 0.        ]), array([0.99907194, 0.        ]), array([0.96219815, 0.        ]), array([62496,  4848]))
[[62438    58]
 [ 4848     0]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
(array([0.95685326, 0.14285714]), array([0.99883518, 0.00429185]), array([0.97739362, 0.00833333]), array([5151,  233]))
[[5145    6]
 [ 232    1]]


***
***
# Traditional Learner Application
## Regular Datasets VS SMOTE Datasets
***
***

***
### Bio -> Astro 
***

In [66]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

(array([0.92764458, 0.06862152]), array([0.90140169, 0.09364686]), array([0.91433487, 0.07920447]), array([62496,  4848]))
[[56334  6162]
 [ 4394   454]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92683867, 0.06701432]), array([0.80819572, 0.17759901]), array([0.86346075, 0.09731013]), array([62496,  4848]))
[[50509 11987]
 [ 3987   861]]


In [67]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

(array([0.92802218, 0.07313196]), array([0.9906714 , 0.00948845]), array([0.95832398, 0.01679752]), array([62496,  4848]))
[[61913   583]
 [ 4802    46]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92871982, 0.07420343]), array([0.75824053, 0.24979373]), array([0.8348661 , 0.11441799]), array([62496,  4848]))
[[47387 15109]
 [ 3637  1211]]


In [68]:
clf = BernoulliNB()

run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)  

(array([0.9280114, 0.       ]), array([1., 0.]), array([0.96266174, 0.        ]), array([62496,  4848]))
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94785276, 0.07617609]), array([0.17799539, 0.87376238]), array([0.29970902, 0.14013497]), array([62496,  4848]))
[[11124 51372]
 [  612  4236]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [69]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

(array([0.9280817 , 0.07304846]), array([0.93786802, 0.06311881]), array([0.9329492 , 0.06772159]), array([62496,  4848]))
[[58613  3883]
 [ 4542   306]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92940264, 0.07401862]), array([0.59424603, 0.41811056]), array([0.72496218, 0.12577172]), array([62496,  4848]))
[[37138 25358]
 [ 2821  2027]]


In [70]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9280114, 0.       ]), array([1., 0.]), array([0.96266174, 0.        ]), array([62496,  4848]))
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92622174, 0.06951664]), array([0.57893305, 0.40552805]), array([0.71251194, 0.11868755]), array([62496,  4848]))
[[36181 26315]
 [ 2882  1966]]


***
### Bio -> Chem
***

In [71]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

(array([0.9429508 , 0.05810109]), array([0.8385327 , 0.16410941]), array([0.88768163, 0.08581894]), array([49397,  2998]))
[[41421  7976]
 [ 2506   492]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94474254, 0.06804329]), array([0.84833087, 0.18245497]), array([0.89394473, 0.09912114]), array([49397,  2998]))
[[41905  7492]
 [ 2451   547]]


In [72]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94321608, 0.05943122]), array([0.8359617 , 0.17078052]), array([0.88635607, 0.08817704]), array([49397,  2998]))
[[41294  8103]
 [ 2486   512]]


In [73]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

(array([0.94261089, 0.04575163]), array([0.98522177, 0.01167445]), array([0.96344542, 0.01860218]), array([49397,  2998]))
[[48667   730]
 [ 2963    35]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94038426, 0.05470887]), array([0.51029415, 0.46697799]), array([0.66158369, 0.09794319]), array([49397,  2998]))
[[25207 24190]
 [ 1598  1400]]


In [74]:
clf = BernoulliNB()

run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94835025, 0.0582051 ]), array([0.15128449, 0.86424283]), array([0.26094244, 0.10906489]), array([49397,  2998]))
[[ 7473 41924]
 [  407  2591]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [75]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94439391, 0.06356531]), array([0.79869223, 0.2251501 ]), array([0.86545359, 0.09914078]), array([49397,  2998]))
[[39453  9944]
 [ 2323   675]]


***
### Bio -> Others
***

In [76]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

(array([0.95508529, 0.03320053]), array([0.85866822, 0.10729614]), array([0.90431405, 0.05070994]), array([5151,  233]))
[[4423  728]
 [ 208   25]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.957994  , 0.05153203]), array([0.86779266, 0.15879828]), array([0.91066517, 0.07781283]), array([5151,  233]))
[[4470  681]
 [ 196   37]]


In [77]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95826162, 0.05298913]), array([0.86468647, 0.16738197]), array([0.90907235, 0.08049536]), array([5151,  233]))
[[4454  697]
 [ 194   39]]


In [78]:
clf = BernoulliNB()

run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.97780374, 0.04726148]), array([0.16249272, 0.91845494]), array([0.27867488, 0.08989708]), array([5151,  233]))
[[ 837 4314]
 [  19  214]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [79]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

(array([0.9568006 , 0.04819277]), array([0.98466317, 0.01716738]), array([0.97053196, 0.02531646]), array([5151,  233]))
[[5072   79]
 [ 229    4]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95987159, 0.05042527]), array([0.69656377, 0.35622318]), array([0.80728991, 0.08834486]), array([5151,  233]))
[[3588 1563]
 [ 150   83]]


In [80]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.96142633, 0.06594595]), array([0.83226558, 0.26180258]), array([0.89219563, 0.10535406]), array([5151,  233]))
[[4287  864]
 [ 172   61]]


***
### Astro -> Bio
***

In [81]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

(array([0.90793615, 0.0745277 ]), array([0.87138855, 0.10491824]), array([0.88928701, 0.08714954]), array([159216,  15717]))
[[138739  20477]
 [ 14068   1649]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.90930129, 0.08403662]), array([0.87116245, 0.11974295]), array([0.88982339, 0.09876154]), array([159216,  15717]))
[[138703  20513]
 [ 13835   1882]]


In [82]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.90034285, 0.07800416]), array([0.54098834, 0.39339569]), array([0.67586824, 0.13019309]), array([159216,  15717]))
[[86134 73082]
 [ 9534  6183]]


In [83]:
clf = BernoulliNB()

run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92538125, 0.09292735]), array([0.17112602, 0.86021505]), array([0.28883859, 0.16773465]), array([159216,  15717]))
[[ 27246 131970]
 [  2197  13520]]


In [84]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

(array([0.90994659, 0.07553957]), array([0.98547257, 0.0120252 ]), array([0.94620485, 0.02074757]), array([159216,  15717]))
[[156903   2313]
 [ 15528    189]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.9075087 , 0.08732357]), array([0.48665963, 0.49755042]), array([0.6335646 , 0.14857175]), array([159216,  15717]))
[[77484 81732]
 [ 7897  7820]]


In [85]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.9009468, 0.0843795]), array([0.36875691, 0.58929821]), array([0.5233195 , 0.14762159]), array([159216,  15717]))
[[ 58712 100504]
 [  6455   9262]]


***
### Astro -> Chem
***

In [86]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

(array([0.94248297, 0.05605905]), array([0.79547341, 0.20013342]), array([0.8627606 , 0.08758485]), array([49397,  2998]))
[[39294 10103]
 [ 2398   600]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94325998, 0.0596479 ]), array([0.83563779, 0.17178119]), array([0.88619335, 0.08854883]), array([49397,  2998]))
[[41278  8119]
 [ 2483   515]]


In [87]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94736306, 0.06309918]), array([0.5647509 , 0.48298866]), array([0.70765055, 0.11161643]), array([49397,  2998]))
[[27897 21500]
 [ 1550  1448]]


In [88]:
clf = BernoulliNB()

run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94835025, 0.0582051 ]), array([0.15128449, 0.86424283]), array([0.26094244, 0.10906489]), array([49397,  2998]))
[[ 7473 41924]
 [  407  2591]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [89]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

(array([0.94118014, 0.03946457]), array([0.91574387, 0.05703803]), array([0.92828779, 0.04665121]), array([49397,  2998]))
[[45235  4162]
 [ 2827   171]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94471847, 0.05804154]), array([0.29856064, 0.71214143]), array([0.45372877, 0.10733498]), array([49397,  2998]))
[[14748 34649]
 [  863  2135]]


In [90]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9427808, 0.       ]), array([1., 0.]), array([0.97054778, 0.        ]), array([49397,  2998]))
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95305203, 0.0602678 ]), array([0.23137033, 0.81220814]), array([0.37234684, 0.1122094 ]), array([49397,  2998]))
[[11429 37968]
 [  563  2435]]


***
### Astro -> Others
***

In [91]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

(array([0.95488722, 0.03634752]), array([0.78897301, 0.17596567]), array([0.86403742, 0.06024982]), array([5151,  233]))
[[4064 1087]
 [ 192   41]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95722041, 0.04621309]), array([0.85575616, 0.15450644]), array([0.90364904, 0.07114625]), array([5151,  233]))
[[4408  743]
 [ 197   36]]


In [92]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95052807, 0.04016736]), array([0.33197437, 0.61802575]), array([0.49208633, 0.07543216]), array([5151,  233]))
[[1710 3441]
 [  89  144]]


In [93]:
clf = BernoulliNB()

run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.97780374, 0.04726148]), array([0.16249272, 0.91845494]), array([0.27867488, 0.08989708]), array([5151,  233]))
[[ 837 4314]
 [  19  214]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [94]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

(array([0.95702385, 0.05882353]), array([0.98136284, 0.02575107]), array([0.96904054, 0.0358209 ]), array([5151,  233]))
[[5055   96]
 [ 227    6]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95969155, 0.04662189]), array([0.53154727, 0.50643777]), array([0.68415792, 0.0853835 ]), array([5151,  233]))
[[2738 2413]
 [ 115  118]]


In [95]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.98011138, 0.05039981]), array([0.23917686, 0.89270386]), array([0.38451935, 0.09541284]), array([5151,  233]))
[[1232 3919]
 [  25  208]]


***
### Chem -> Bio
***

In [96]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

(array([0.90885843, 0.07519696]), array([0.91742664, 0.06801552]), array([0.91312244, 0.07142619]), array([159216,  15717]))
[[146069  13147]
 [ 14648   1069]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.90927017, 0.08147261]), array([0.90362778, 0.08659413]), array([0.90644019, 0.08395534]), array([159216,  15717]))
[[143872  15344]
 [ 14356   1361]]


In [97]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.90834076, 0.08677997]), array([0.6270915 , 0.35897436]), array([0.74195754, 0.13977109]), array([159216,  15717]))
[[99843 59373]
 [10075  5642]]


In [98]:
clf = BernoulliNB()

run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92538125, 0.09292735]), array([0.17112602, 0.86021505]), array([0.28883859, 0.16773465]), array([159216,  15717]))
[[ 27246 131970]
 [  2197  13520]]


In [99]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

(array([0.90859046, 0.042814  ]), array([0.96615918, 0.01533372]), array([0.93649093, 0.02258034]), array([159216,  15717]))
[[153828   5388]
 [ 15476    241]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.90017386, 0.06538848]), array([0.70240428, 0.21091811]), array([0.78908598, 0.09982835]), array([159216,  15717]))
[[111834  47382]
 [ 12402   3315]]


In [100]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.91015417, 0.        ]), array([1., 0.]), array([0.9529641, 0.       ]), array([159216,  15717]))
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.9054337 , 0.08197071]), array([0.62198523, 0.34192276]), array([0.7374092 , 0.13223913]), array([159216,  15717]))
[[99030 60186]
 [10343  5374]]


***
### Chem -> Astro
***

In [101]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

(array([0.92789231, 0.07046015]), array([0.92759537, 0.07075083]), array([0.92774381, 0.07060519]), array([62496,  4848]))
[[57971  4525]
 [ 4505   343]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92843541, 0.07567257]), array([0.89719342, 0.10849835]), array([0.9125471, 0.0891601]), array([62496,  4848]))
[[56071  6425]
 [ 4322   526]]


In [102]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9280114, 0.       ]), array([1., 0.]), array([0.96266174, 0.        ]), array([62496,  4848]))
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.93220599, 0.07781246]), array([0.58394137, 0.45255776]), array([0.71807485, 0.13279264]), array([62496,  4848]))
[[36494 26002]
 [ 2654  2194]]


In [103]:
clf = BernoulliNB()

run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

(array([0.9280114, 0.       ]), array([1., 0.]), array([0.96266174, 0.        ]), array([62496,  4848]))
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94785276, 0.07617609]), array([0.17799539, 0.87376238]), array([0.29970902, 0.14013497]), array([62496,  4848]))
[[11124 51372]
 [  612  4236]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [104]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

(array([0.92731704, 0.03643411]), array([0.98011073, 0.00969472]), array([0.95298327, 0.01531443]), array([62496,  4848]))
[[61253  1243]
 [ 4801    47]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.92855866, 0.07360989]), array([0.74807988, 0.25804455]), array([0.82860561, 0.11454471]), array([62496,  4848]))
[[46752 15744]
 [ 3597  1251]]


In [105]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.9280114, 0.       ]), array([1., 0.]), array([0.96266174, 0.        ]), array([62496,  4848]))
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.93793936, 0.08494182]), array([0.57216462, 0.5119637 ]), array([0.71075333, 0.14570858]), array([62496,  4848]))
[[35758 26738]
 [ 2366  2482]]


***
### Chem -> Others
***

In [106]:
clf = DecisionTreeClassifier(random_state=37)

run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

(array([0.9576461 , 0.05285412]), array([0.9130266 , 0.10729614]), array([0.93480421, 0.07082153]), array([5151,  233]))
[[4703  448]
 [ 208   25]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.96010266, 0.0564663 ]), array([0.79887401, 0.26609442]), array([0.87209918, 0.09316304]), array([5151,  233]))
[[4115 1036]
 [ 171   62]]


In [107]:
clf = KNeighborsClassifier(n_neighbors = 547)

run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.94930556, 0.04056795]), array([0.26538536, 0.68669528]), array([0.41480807, 0.07661001]), array([5151,  233]))
[[1367 3784]
 [  73  160]]


In [108]:
clf = BernoulliNB()

run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.97780374, 0.04726148]), array([0.16249272, 0.91845494]), array([0.27867488, 0.08989708]), array([5151,  233]))
[[ 837 4314]
 [  19  214]]


/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [109]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

(array([0.95685612, 0.05660377]), array([0.99029315, 0.01287554]), array([0.97328754, 0.02097902]), array([5151,  233]))
[[5101   50]
 [ 230    3]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.95846154, 0.04784367]), array([0.72568433, 0.30472103]), array([0.82598608, 0.08270239]), array([5151,  233]))
[[3738 1413]
 [ 162   71]]


In [110]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

/home/tntech.edu/ambecklehi42/.conda/envs/tf-2.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.95672363, 0.        ]), array([1., 0.]), array([0.97788325, 0.        ]), array([5151,  233]))
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(array([0.97539016, 0.05164067]), array([0.31547272, 0.82403433]), array([0.4767493 , 0.09719058]), array([5151,  233]))
[[1625 3526]
 [  41  192]]
